# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Api_keys import api_key_GoogleMaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
data_path = "Output/WeatherPyOutput.csv"

main_data_df = pd.read_csv(data_path)

main_data_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp (C),Wind Speed
0,0,Torbay,90,CA,1612258469,86,47.6666,-52.7314,-2.78,3.60
1,1,Alta Floresta,42,BR,1612258470,97,-9.8756,-56.0861,21.23,0.92
2,2,Katsuura,40,JP,1612258423,43,35.1333,140.3000,11.00,8.75
3,3,Vao,100,NC,1612258473,85,-22.6667,167.4833,24.25,13.65
4,4,Comodoro Rivadavia,0,AR,1612258474,48,-45.8667,-67.5000,19.00,2.06
...,...,...,...,...,...,...,...,...,...,...
669,669,Patti,0,IN,1612259298,49,31.2808,74.8572,22.00,3.09
670,670,Mountain Home,40,US,1612259337,60,43.1330,-115.6912,3.00,7.20
671,671,Los Llanos de Aridane,75,ES,1612259338,82,28.6585,-17.9182,18.00,3.60
672,672,La Rioja,20,ES,1612259340,63,42.2500,-2.5000,15.00,3.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=api_key_GoogleMaps)

cities_coords_df = main_data_df[["Latitude", "Longitude"]]
humidity_df = main_data_df['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(cities_coords_df, weights=humidity_df, 
                                 dissipating=False, max_intensity=max(main_data_df["Humidity"]),
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
weather_fit_df = main_data_df.copy()

weather_fit_df = weather_fit_df.loc[(weather_fit_df["Max Temp (C)"] < 40) & (weather_fit_df["Max Temp (C)"] > 25)]
weather_fit_df = weather_fit_df.loc[(weather_fit_df["Wind Speed"] < 10) & (weather_fit_df["Wind Speed"] > 5)]
weather_fit_df = weather_fit_df.loc[(weather_fit_df["Cloudiness"] < 10) & (weather_fit_df["Cloudiness"] > 3)]

weather_fit_clean_df = weather_fit_df.drop(columns="Unnamed: 0")

weather_fit_clean_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp (C),Wind Speed
97,Grand Gaube,8,MU,1612258586,69,-20.0064,57.6608,29.44,5.36
109,Cap Malheureux,6,MU,1612258600,69,-19.9842,57.6142,29.44,5.36
460,Cockburn Town,5,TC,1612259062,75,21.4612,-71.1419,25.57,6.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
hotel_df = weather_fit_clean_df

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

parameters = {
    "radius": 5000,
    "types": "lodging",
    "key": api_key_GoogleMaps
}

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city = row["City"]


    parameters["location"] = f"{latitude},{longitude}"  
    
    print(f"Getting output for Index {index}: {city}.")
    response = requests.get(url, params=parameters).json()
    
    results = response['results']
    name = results[0]["name"]

    try:
        print(f"The closest hotel is {name}.")
        hotel_df.loc[index, "Hotel Name"] = name
        
    except:
        print("City not found. Processing next record.")

Getting output for Index 97: Grand Gaube.
The closest hotel is Veranda Paul et Virginie Hotel & Spa.
Getting output for Index 109: Cap Malheureux.
The closest hotel is Pereybere Hotel & Spa.
Getting output for Index 460: Cockburn Town.
The closest hotel is Osprey Beach Hotel.


In [87]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp (C),Wind Speed,Hotel Name
97,Grand Gaube,8,MU,1612258586,69,-20.0064,57.6608,29.44,5.36,Veranda Paul et Virginie Hotel & Spa
109,Cap Malheureux,6,MU,1612258600,69,-19.9842,57.6142,29.44,5.36,Pereybere Hotel & Spa
460,Cockburn Town,5,TC,1612259062,75,21.4612,-71.1419,25.57,6.35,Osprey Beach Hotel


In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [96]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))